In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import re
import pandas as pd
import pickle
import requests
import deepl

## Load Model Files

In [2]:
model = tf.keras.models.load_model('FairyTaleProject.keras')

2024-02-01 14:21:58.843188: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-01 14:21:58.843222: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-01 14:21:58.843230: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-01 14:21:58.843284: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-01 14:21:58.843309: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
with open('tokenizer.pkl','rb') as file:
    tokenizer = pickle.load(file)

In [4]:
with open('training_history.pkl','rb') as file:
    history = pickle.load(file)

In [5]:
with open('deepl_api.txt','r') as file:
    deepl_api = file.read()

## Gradio Interface

In [6]:
def deepl_translator(text):
    auth_key = deepl_api  # Replace with your key
    translator = deepl.Translator(auth_key)
    
    result = translator.translate_text(text, target_lang="TR")
    return result.text

In [7]:
import gradio as gr

max_sequence_len = 403

def generator(markdown,seed_text,next_words):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word

  translated_text = deepl_translator(seed_text)  
    
  return seed_text,translated_text

demo = gr.Interface(
    fn=generator,
    
    inputs=[
        gr.Markdown('# FairyMairy'),
        gr.Textbox(lines=15),
        gr.Slider(maximum=30,step=1)
        
    ],
    outputs=[
        gr.Textbox(lines=10,label='İngilizce'),
        gr.Textbox(lines=10,label='Türkçe')
    ],
)

demo.launch(share=True)

/Users/yaseminsungur/anaconda3/envs/btk2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on public URL: https://e39c5561c47b827bf1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 23ms/step


2024-02-01 14:22:32.574132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 16ms/step


In [8]:
#demo.close()

Closing server running on port: 7860
